# 📊 Day 15 — Advanced RAG Evaluation Notebook

**Goal:** Systematically compare HyDE, Self-Query, Multi-Query, and Graph RAG  
**Framework:** [RAGAS](https://github.com/explodinggradients/ragas) — industry-standard RAG evaluation metrics  

---
### Metrics We'll Measure

| Metric | What it measures | Ideal value |
|--------|-----------------|-------------|
| **Faithfulness** | Is the answer grounded in retrieved context? (no hallucinations) | > 0.85 |
| **Answer Relevancy** | Does the answer address the question? | > 0.80 |
| **Context Precision** | Are retrieved chunks actually useful? | > 0.75 |
| **Context Recall** | Did we retrieve all relevant chunks? | > 0.70 |
| **Latency** | Time to first token & total response time | < 3s TTFT |

---
**Interview tip:** When asked 'how do you evaluate a RAG system?', cite these metrics and explain the hallucination vs. retrieval tradeoff.

In [ ]:
# ── 0. INSTALL DEPENDENCIES (run once) ──────────────────────────────────────
# !pip install ragas llama-index llama-index-llms-openai llama-index-embeddings-openai \
#             pandas matplotlib seaborn python-dotenv tqdm nest_asyncio

In [ ]:
# ── 1. IMPORTS & CONFIGURATION ───────────────────────────────────────────────
import os
import time
import asyncio
import nest_asyncio
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from dotenv import load_dotenv

nest_asyncio.apply()  # Required to run async code in Jupyter

load_dotenv()
assert os.getenv('OPENAI_API_KEY'), 'Set OPENAI_API_KEY in .env!'

# LlamaIndex
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model='gpt-4o-mini', temperature=0)
Settings.embed_model = OpenAIEmbedding(model='text-embedding-3-small')
Settings.chunk_size = 512

print('✅ Setup complete')

In [ ]:
# ── 2. GROUND-TRUTH EVALUATION DATASET ──────────────────────────────────────
# In production: source these from domain experts or label studio.
# Each entry: question | ground_truth_answer | relevant_document_ids

EVAL_QUESTIONS = [
    {
        'question': 'What is HyDE and why does it improve retrieval?',
        'ground_truth': 'HyDE (Hypothetical Document Embeddings) improves retrieval by generating '
                        'a hypothetical answer to the query, then embedding that answer instead of '
                        'the query itself. This reduces the semantic gap between question and document embeddings.',
    },
    {
        'question': 'How does multi-query retrieval handle duplicate results?',
        'ground_truth': 'Multi-query retrieval uses Reciprocal Rank Fusion (RRF) to deduplicate and '
                        'rerank results from multiple query variants. Nodes appearing in multiple result '
                        'sets receive higher scores.',
    },
    {
        'question': 'What is the difference between vector RAG and graph RAG?',
        'ground_truth': 'Vector RAG uses semantic similarity search to find relevant chunks. '
                        'Graph RAG builds a knowledge graph of entities and relations, enabling '
                        'multi-hop reasoning across connected concepts.',
    },
    {
        'question': 'Why use SSE for streaming RAG responses?',
        'ground_truth': 'Server-Sent Events (SSE) is an HTTP-native, one-way push protocol that '
                        'efficiently streams LLM tokens to clients. It reduces time-to-first-token '
                        'and improves perceived user experience.',
    },
]

print(f'Evaluation dataset: {len(EVAL_QUESTIONS)} questions')

In [ ]:
# ── 3. BUILD INDEX ──────────────────────────────────────────────────────────
DATA_DIR = './data'
os.makedirs(DATA_DIR, exist_ok=True)

# Create demo data if needed
if not os.listdir(DATA_DIR):
    with open(f'{DATA_DIR}/rag_overview.txt', 'w') as f:
        f.write(
            "HyDE or Hypothetical Document Embeddings improves retrieval by generating a hypothetical answer. "
            "Multi-query retrieval uses Reciprocal Rank Fusion to deduplicate results from query variants. "
            "Graph RAG uses knowledge graphs with entity and relation extraction unlike vector RAG. "
            "Server-Sent Events SSE is used for streaming because it is HTTP-native and one-way. "
            "Self-query retrieval lets the LLM generate structured metadata filters before vector search."
        )

documents = SimpleDirectoryReader(DATA_DIR).load_data()
index = VectorStoreIndex.from_documents(documents)
print(f'Index built from {len(documents)} documents')

In [ ]:
# ── 4. DEFINE RETRIEVAL STRATEGY RUNNERS ────────────────────────────────────
from llama_index.core.query_engine import TransformQueryEngine
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.retrievers import QueryFusionRetriever, VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine


def run_strategy(strategy_name: str, question: str) -> dict:
    """
    Runs a single question through the specified RAG strategy.
    Records answer, retrieved contexts, and latency.
    """
    start = time.time()

    if strategy_name == 'standard':
        engine = index.as_query_engine(similarity_top_k=4)
        response = engine.query(question)

    elif strategy_name == 'hyde':
        base_engine = index.as_query_engine(similarity_top_k=4)
        hyde = HyDEQueryTransform(include_original=True)
        engine = TransformQueryEngine(base_engine, hyde)
        response = engine.query(question)

    elif strategy_name == 'multi_query':
        retriever = QueryFusionRetriever(
            retrievers=[VectorIndexRetriever(index=index, similarity_top_k=4)],
            num_queries=4,
            mode='reciprocal_rerank',
            use_async=False,
        )
        engine = RetrieverQueryEngine.from_args(retriever)
        response = engine.query(question)

    else:
        raise ValueError(f'Unknown strategy: {strategy_name}')

    latency = time.time() - start

    return {
        'strategy': strategy_name,
        'question': question,
        'answer': str(response),
        'contexts': [n.text for n in response.source_nodes] if hasattr(response, 'source_nodes') else [],
        'latency_s': round(latency, 2),
    }


print('Strategy runners defined')

In [ ]:
# ── 5. RUN EVALUATION ────────────────────────────────────────────────────────
STRATEGIES = ['standard', 'hyde', 'multi_query']
results = []

for strategy in STRATEGIES:
    print(f'\nEvaluating strategy: {strategy}')
    for item in tqdm(EVAL_QUESTIONS):
        result = run_strategy(strategy, item['question'])
        result['ground_truth'] = item['ground_truth']
        results.append(result)

df_results = pd.DataFrame(results)
print(f'\n✅ Evaluation complete: {len(df_results)} results')
df_results[['strategy', 'question', 'latency_s', 'answer']].head()

In [ ]:
# ── 6. RAGAS AUTOMATED SCORING ───────────────────────────────────────────────
# RAGAS evaluates RAG pipelines using LLM-as-judge methodology.
# Reference: https://docs.ragas.io/en/latest/

from ragas import evaluate
from ragas.metrics import (
    faithfulness,          # Is answer grounded in context? Penalises hallucination.
    answer_relevancy,      # Does answer address the question?
    context_precision,     # Are retrieved chunks relevant?
    context_recall,        # Did we retrieve all needed chunks?
)
from datasets import Dataset

# Build RAGAS-compatible dataset format
ragas_data = {
    'question':   [r['question']    for r in results],
    'answer':     [r['answer']      for r in results],
    'contexts':   [r['contexts']    for r in results],
    'ground_truth': [r['ground_truth'] for r in results],
}
ragas_dataset = Dataset.from_dict(ragas_data)

# Run RAGAS evaluation — this calls the LLM internally to score each sample
ragas_scores = evaluate(
    ragas_dataset,
    metrics=[faithfulness, answer_relevancy, context_precision, context_recall],
)

df_ragas = ragas_scores.to_pandas()
df_ragas['strategy'] = [r['strategy'] for r in results]
print('RAGAS evaluation complete!')
df_ragas.head()

In [ ]:
# ── 7. AGGREGATE SCORES BY STRATEGY ─────────────────────────────────────────
metric_cols = ['faithfulness', 'answer_relevancy', 'context_precision', 'context_recall']
df_grouped = df_ragas.groupby('strategy')[metric_cols].mean().round(3)

# Add latency
latency_df = df_results.groupby('strategy')['latency_s'].mean().round(2)
df_grouped['avg_latency_s'] = latency_df

print('\n📊 Strategy Comparison:')
display(df_grouped.style.background_gradient(cmap='RdYlGn', axis=None))

In [ ]:
# ── 8. VISUALISATION ─────────────────────────────────────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('Advanced RAG Strategy Comparison — Day 15', fontsize=14, fontweight='bold')

# Plot 1: Metric heatmap
sns.heatmap(
    df_grouped[metric_cols],
    ax=axes[0],
    annot=True,
    fmt='.3f',
    cmap='RdYlGn',
    vmin=0,
    vmax=1,
    linewidths=0.5,
)
axes[0].set_title('RAGAS Metrics by Strategy')
axes[0].set_ylabel('')

# Plot 2: Latency vs. quality radar (simplified bar)
strategies = df_grouped.index.tolist()
overall_quality = df_grouped[metric_cols].mean(axis=1)
latencies = df_grouped['avg_latency_s']

x = range(len(strategies))
bars1 = axes[1].bar([i - 0.2 for i in x], overall_quality, 0.35,
                     label='Avg Quality Score', color='#4EA8DE', alpha=0.85)
ax2 = axes[1].twinx()
bars2 = ax2.bar([i + 0.2 for i in x], latencies, 0.35,
                 label='Avg Latency (s)', color='#F6AD55', alpha=0.85)

axes[1].set_xticks(x)
axes[1].set_xticklabels(strategies)
axes[1].set_ylabel('Quality Score (0-1)', color='#4EA8DE')
ax2.set_ylabel('Latency (seconds)', color='#F6AD55')
axes[1].set_title('Quality vs. Latency Trade-off')
axes[1].legend(loc='upper left')
ax2.legend(loc='upper right')

plt.tight_layout()
plt.savefig('rag_evaluation_results.png', dpi=150, bbox_inches='tight')
plt.show()
print('\n✅ Chart saved to rag_evaluation_results.png')

In [ ]:
# ── 9. CUSTOM HALLUCINATION CHECK ────────────────────────────────────────────
# A simple LLM-as-judge prompt for detecting hallucinations.
# More interpretable than black-box RAGAS scores for demo/interview purposes.

from llama_index.llms.openai import OpenAI

llm = OpenAI(model='gpt-4o-mini', temperature=0)

HALLUCINATION_PROMPT = """
You are a fact-checker. Given a context and an answer, determine if the answer 
contains any claims NOT supported by the context (hallucinations).

Context:
{context}

Answer:
{answer}

Respond with JSON only:
{{"hallucination_detected": true/false, "explanation": "brief reason", "confidence": 0.0-1.0}}
"""

def check_hallucination(context: str, answer: str) -> dict:
    import json, re
    prompt = HALLUCINATION_PROMPT.format(context=context[:1500], answer=answer[:500])
    raw = llm.complete(prompt).text.strip()
    raw = re.sub(r'```(?:json)?|```', '', raw).strip()
    try:
        return json.loads(raw)
    except:
        return {'hallucination_detected': None, 'explanation': 'parse_error', 'confidence': 0}


# Run hallucination checks on first question for each strategy
print('\n🔍 Hallucination Check Results:')
for strategy in STRATEGIES:
    row = df_results[(df_results.strategy == strategy)].iloc[0]
    ctx = ' '.join(row['contexts'])[:1000]
    result = check_hallucination(ctx, row['answer'])
    status = '🟢 Clean' if not result.get('hallucination_detected') else '🔴 HALLUCINATION'
    print(f'  [{strategy:12s}] {status} | Confidence: {result.get("confidence",0):.2f} | {result.get("explanation","")[:80]}')

In [ ]:
# ── 10. INTERVIEW SUMMARY ────────────────────────────────────────────────────
print('='*65)
print('📋  INTERVIEW-READY SUMMARY: WHEN TO USE EACH STRATEGY')
print('='*65)

summary = {
    'Standard RAG':  'Best for simple factual queries, lowest latency, baseline',
    'HyDE':          'Best for zero-shot domains, improves recall by 15-30%, adds ~0.5s latency',
    'Multi-Query':   'Best for ambiguous queries, highest recall, adds 2-4x LLM calls',
    'Graph RAG':     'Best for multi-hop relational queries, requires graph construction overhead',
    'Self-Query':    'Best when metadata filtering is needed (e.g., date, author, category)',
}

for strategy, use_case in summary.items():
    print(f'  {strategy:15s} → {use_case}')

print('\n📊 Key Metric Targets:')
print('  Faithfulness > 0.85 (hallucination guard)')
print('  Context Recall > 0.70 (retrieval completeness)')
print('  TTFT < 500ms (streaming UX threshold)')
print('\n✅ Day 15 evaluation complete!')